In [1]:
import random
import numpy as np
import torch
import torch.nn as nn

from models import *

/home/local/VANDERBILT/lij112/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

batch, length, dim = 2, 64, 16
x = torch.randn(batch, length, dim).cuda()

model = Mamba2Block(
    d_model=dim,
    d_state=16,
    d_conv=4,
    expand=2,
    headdim=32,
).cuda()

with torch.no_grad():
    y = model(x)

print("Output shape:", y.shape)

Output shape: torch.Size([64, 16])
